In [1]:
from models import Finetuner
import optuna
import sys
sys.path.insert(1, "../BenchCanc/")
from PyDRP.Benchmarks import BenchCanc

In [2]:
args = {"model_file": "attn_gnn",
        "pooling":"mean",
        "pretraining":"properties2d",
         "study":"properties2d_attn_gnn_0"}
config = {"model_file":args["model_file"],
          "transfer_decoder":True,
                  "pooling":args["pooling"],
                  "pretraining_epoch":0,
                  "freeze":"partial+gnn",
                  "num_epochs":51,
                  "pretraining":args["pretraining"],
                 "device":"cuda:7"}
if args["study"] == "none":
    config_model = {"embed_dim":250,
              "hidden_dim":1024,
              "learning_rate":0.0001,
              "num_layers":4}
else:
    study_name = args["study"]
    storage_name = "sqlite:///studies/{}.db".format(study_name)
    study = optuna.load_study(study_name, storage_name)
    config_model = study.best_params
    config_model["embed_dim"]*=60
config = {**config_model, **config}
model_name = config["model_file"].replace('.py', '')
target_model = __import__(model_name)
lr = config["learning_rate"]
pretraining = config["pretraining"]
if "pooling" in config.keys():
    pooling = config["pooling"]
else:
    pooling = "mean"
config_model2d = target_model.config_model2d
config_model2d = {**config, **config_model2d}

In [3]:
model = Finetuner(model2D = target_model.Model2D(train_dataset= None,**config_model2d),
               init_dim=79,
               embed_dim=config["embed_dim"],
               edge_dim = 10,
               pooling = pooling,
               hidden_dim = config["hidden_dim"])

In [4]:
model.load_from_pretrained(f'models/attn_gnn_{config["pretraining"]}_{config["pooling"]}_{config["pretraining_epoch"]}.pt')
if config["transfer_decoder"]:
    model.transfer_mlp()
if not config["freeze"]:
    pass
else:
    if "gnn" in config["freeze"]:
        model.freeze_gnn()
    if "pooling" in config["freeze"]:
        model.pooling.freeze()
    if "partial" in config["freeze"]:
        model.partial_unfreeze()

In [5]:
config = {"optimizer":{"batch_size":256,
                        "learning_rate":0.0001,
                         "max_epochs":100,
                         "patience":4,
                         "clip_norm":10,
                         "kwargs":{}},
            "env":{"device":config["device"],
                   "mixed_precision":True},
            "model":config}
def epoch_callback(epoch, model, optimizer, scheduler, train_metrics, test_metrics, config):
    if epoch == config["model"]["pretraining_length"]:
        model.set_warm()
    test_metrics = {it[0]:it[1].item() for it in test_metrics.compute().items()}
    print(f"epoch : {epoch}, test_metrics: {test_metrics}")
ps = []

In [ ]:
benchmark = BenchCanc(config,
                    minmax_target=False,
                    epoch_callback = epoch_callback,
                    line_features = "expression+mutations+cnv",
                    setting = "precision_oncology",
                    residualize=True,
                    dataset = "GDSC1",)
benchmark.train_model(model, 0)